<a href="https://colab.research.google.com/github/Ikochoy/Mix-MatchImageCaptioning/blob/main/CSC413_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# linear algebra
import numpy as np  
# data processing, CSV file I / O (e.g. pd.read_csv)
import pandas as pd  
import os
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt  # for plotting data
import cv2
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from PIL import Image
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm
import concurrent.futures
from torch import nn

In [ ]:
CROPPED_WIDTH = 299
CROPPED_HEIGHT = 299

# DataSets Preprocessing 

In [ ]:
class Image:
  def __init__(self, image_id, annotations=[], file_name=None):
    self.image_id = image_id
    self.annotations = annotations
    self.file_name = file_name
  
  def add_annotation(self, annotation):
    self.annotations.append(annotation)

  def get_annotations(self):
    return self.annotations

  def __str__(self):
    idx = f"{self.image_id}\n" 
    annotations = "\n".join(self.annotations)
    file_url = f"{self.file_name}"
    return idx + annotations + file_url

class ImageSet:
  def __init__(self, images={}):
    # Dictionary of images
    images
    self.images = images
  
  def add_image(self,image_id, image):
    self.images[image_id] = image
  
  def get_image_by_id(self, image_id):
    return self.images[image_id]
  
  def get_image_list(self):



class DataSet:
  def __init__(self, images):
    self.train = ImageSet()
    self.validation = ImageSet()
  
  def add_training_image(self, image):
    self.train.add_image(image)
  
  def add_validation_image(self, image):
    self.validation.add_image(self)



Load Images From Urls

In [ ]:
def download_image(path, image_url, image_id):
  response = requests.get(image_url)
  img = response.content
  if f'{image_id}.jpg' not in os.listdir(path):
      with open(path + "/" + f'{image_id}' + '.jpg', 'wb') as handler:
        handler.write(img)


def load_images_to_dir(path, imageset):
  # load images concurrently
  with concurrent.futures.ThreadPoolExecutor(
        max_workers=8
    ) as executor:
        future_to_url = {
            executor.submit(download_image, path, image.file_name, image_id): image.file_name
            for image_id, image in imageset.images.items()
        }
        for future in tqdm(concurrent.futures.as_completed(
            future_to_url
        )):
            url = future_to_url[future]
            try:
                future.result()
            except Exception as exc:
                print(
                    "%r generated an exception: %s" % (url, exc)
                )

## Microsoft COCO

In [ ]:
#!unzip /content/drive/MyDrive/Important_Undergrad_Courses/csc413/captions_train-val2014.zip -d /content/drive/MyDrive/Important_Undergrad_Courses/csc413

In [ ]:
! mkdir coco_train
! mkdir coco_valid

In [ ]:
valid = json.load(open('/content/drive/MyDrive/Important_Undergrad_Courses/csc413/annotations/captions_val2014.json', 'r'))
train = json.load(open('/content/drive/MyDrive/Important_Undergrad_Courses/csc413/annotations/captions_train2014.json', 'r'))

FileNotFoundError: ignored

In [ ]:
print(train["images"][0])
print(train["annotations"][0])

In [ ]:
def get_intermediate_dict(data):
  result = {}
  for annotation in data["annotations"]:
    image_id = annotation["image_id"]
    if image_id in result:
      result[image_id].add_annotation(annotation["caption"])
    else:
      result[image_id] = Image(image_id, [annotation["caption"]])
  for image in data["images"]:
    if image['id'] not in result:
      continue
    else:
      result[image['id']].file_name = image['flickr_url']
  return result

coco_train = ImageSet(get_intermediate_dict(train))
coco_valid = ImageSet(get_intermediate_dict(valid))


In [ ]:
url = list(coco_train.images.values())[0].file_name

In [ ]:
response = requests.get(url, stream=True)
img = Image.open(response.raw)
img = img.resize((224, 224))
plt.imshow(img)
plt.show()

In [ ]:
COCO_TRAIN = "/content/coco_train"
COCO_VALID = "/content/coco_valid"
# load image to coco_train dir

    #plt.imsave(img, f"{path}/{image_id}.jpg")

In [ ]:
# load training images 
load_images_to_dir(COCO_TRAIN, coco_train)
# load validation images
load_images_to_dir(COCO_VALID, coco_valid)

## Flickr30k

In [ ]:
! pip install kaggle

In [ ]:
! #mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download adityajn105/flickr30k

In [ ]:
! unzip flickr30k.zip

In [ ]:
# load txt file
flickr30k_images = {}
with open('/content/captions.txt', 'r') as f:
    f.readline()
    for line in f:
      text_data = line.split(",")
      image_id = text_data[0][:-4]
      file_name = text_data[0]
      caption = text_data[1]
      if image_id in flickr30k_images:
        flickr30k_images[image_id].add_annotation(caption)
      else:
        flickr30k_images[image_id] = Image(image_id, [caption], file_name)
flickr30k_dataset = ImageSet(flickr30k_images)

## Google Conceptual Image Captioning Dataset


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Important_Undergrad_Courses/csc413/googletrainingcaptions.tsv", sep='\t', header=None)

In [ ]:
! mkdir google_conceptual

In [ ]:
data.columns = ["caption", "image_url"]

In [ ]:
data

In [ ]:
images = {}
for idx, row in data.iterrows():
  images[str(idx)] = Image(str(idx), annotations=[row["caption"]], file_name=row["image_url"])

In [ ]:
google_set = ImageSet(images)

In [ ]:
GOOGLE = "/content/google_conceptual"
load_images_to_dir(GOOGLE, google_set)

## Image Preprocessing

What do we need to do in image preprocessing?
- resize image (crop images)
- smooth image a bit
- sharpen edge a bit?

## Captions Preprocessing

# Models

## Part 1: Object Detection -- Transfer Learning

In [ ]:
## GoogLeNet
import torch
from torchvision import transforms
from functorch import combine_state_for_ensemble, vmap

In [ ]:
class ImageEncoder:
  def __init__(self, choice):
    # TODO: Define feature_extract

    self.input = CROPPED_WIDTH
    # Detect if we have a GPU available
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.trained = False
    self.dropout = nn.Dropout(p=0.5)
    self.ensemble = None
    if choice == "GoogLeNet":
      # GoogLeNet
      self.model_name = "GoogLeNet_Inception_v3"
      model_ft = models.inception_v3(pretrained=True)

      # Check whether we need this: Handle the auxilary net
      num_ftrs = model_ft.AuxLogits.fc.in_features
      model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

      # Handle the primary net
      num_ftrs = model_ft.fc.in_features
      model_ft.fc = nn.Linear(num_ftrs,num_classes)
      
      self.transform = transforms.Compose([
          transforms.Resize(224),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      ])
      model_ft = model_ft.to(device)
      self.model = model_ft

    elif choice == "AlexNet":
      self.model_name = "AlexNet"
      model_ft = models.alexnet(pretrained=True)
      self.model = model_ft.to(self.device)

    #TODO
    elif choice == "VGG-19":
      self.model_name = "VGG-19"
      model_ft = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
      self.model = model_ft.to(self.device)

  def generate_ensembling_models(self, num_models):
    # https://pytorch.org/functorch/0.1.0/notebooks/ensembling.html
    models = [self.model for _ in range(num_models)]
    fmodel, params, buffers = combine_state_for_ensemble(models)
    self.ensemble = [fmodel, params, buffers]
  
  def train_and_val_model(self,num_epochs, trainloader, validloader, lr=0.001, momentum=0.9, loss="CE"):
    """https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html"""
    save_file_name = f"{self.model_name}_{num_epochs}_{lr}_{momentum}"
    early_stopping = 5
    # use SGD with learning rate = 0.001 and momentum = 0.9
    self.optimizer = optim.SGD(self.model.parameters(), lr=lr, momentum=momentum)
    if loss=="CE":
      criterion = nn.CrossEntropyLoss()
    elif loss=="L1":
      criterion = nn.L1Loss()
    else:
      criterion = nn.MSELoss()
    valid_loss_min = np.Inf
    epochs_no_improve, valid_best_acc = 0, 0
    outputs_losses = []
    losses_colums = ["training_loss","validation_loss"]
    outputs_acc = []
    acc_columns = ["training_acc","validation_acc"]
    for epoch in tqdm(range(num_epochs)):
      training_loss, validation_loss, testing_loss = 0, 0, 0
      training_acc, validation_acc, testing_acc = 0, 0, 0
      for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        self.optimizer.zero_grad()
        output = self.model(images)
        loss = criterion(output, labels)
        loss.backward()
        self.optimizer.step()
        training_loss += loss.item() * images.size(0)
        # Get training accuracy
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(labels.data.view_as(pred))
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
        training_acc += accuracy.item() * images.size(0)
      else:
        with torch.no_grad():
          self.model.eval()
          print("model start validating")
          for images, labels in validloader:
            images = images.view(images.shape[0], -1)
            output = self.model(images)
            loss = criterion(output, labels)
            validation_loss += loss.item() * images.size(0)
            # Get Validation accuracy
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(labels.data.view_as(pred))
            accuracy = torch.mean(
                correct_tensor.type(torch.FloatTensor))
            validation_acc += accuracy.item() * images.size(0)
       # Calculate average losses
        train_loss = training_loss / len(trainloader.dataset)
        valid_loss = validation_loss / len(validloader.dataset)
        # Calculate average accuracy
        train_acc = training_acc / len(trainloader.dataset)
        valid_acc = validation_acc / len(validloader.dataset)
        outputs_losses.append([
            train_loss, valid_loss
        ])
        outputs_acc.append([
            train_acc, valid_acc
        ])
        # Save the model if validation loss decreases
        if valid_loss < valid_loss_min:
            # Save model
            torch.save(self.model.state_dict(), save_file_name)
            # Track improvement
            epochs_no_improve = 0
            valid_loss_min = valid_loss

        else:
            epochs_no_improve += 1
            if epochs_no_improve >= early_stopping:
                print("Early stopping the model.")
                self.model.load_state_dict(torch.load(save_file_name))
                self.model.optimizer = self.optimizer
                outputs_losses = pd.DataFrame(
                    outputs_losses,
                    columns=losses_colums
                )
                outputs_acc = pd.DataFrame(
                    outputs_acc,
                    columns=acc_columns
                )
                # self.model = model
                self.trained = True
                return self.model, outputs_losses, outputs_acc
    outputs_losses = pd.DataFrame(
        outputs_losses,
        columns=losses_colums
    )
    outputs_acc = pd.DataFrame(
        outputs_acc,
        columns=acc_columns
    )
    # self.model = model
    self.trained = True
    return self.model, outputs_losses, outputs_acc
  
  def forward(self, images, num_models, dropout=False, ensemble=False):
    # think about the ensemble and dropout a bit more
    if ensemble and self.ensemble == None:
      self.generate_ensembling_models(num_models)
    if dropout and ensemble:
      minibatches = images[:num_models]
      model_outputs = self.dropout(vmap(self.ensemble[0])(self.ensemble[1], self.ensemble[2], minibatches))
    elif dropout:
      model_outputs = self.dropout(self.model(images))
    elif ensemble:
      minibatches = images[:num_models]
      model_outputs = vmap(self.ensemble[0])(self.ensemble[1], self.ensemble[2], minibatches)
    else:
      model_outputs = self.model(images)
    # might not want embeddings?
    #embeddings = self.embed(model_outputs)
    return model_outputs

  def encode_image(self, image_path, normalize=False):
    """
    have to make this so that this can also process a lot of images at the same time
    """
    img = Image.open(image_path)
    img_processed = self.transform(img).unsqueeze(0)
    if torch.cuda.is_available():
      img_processed = img_processed.to('cuda')
      self.model.to('cuda')

    if not self.trained:
      print("Model has not been trained")
      return -1
    with torch.no_grad():
      output = self.model(img_processed)[0]
    if normalize:
      # add normalization code here
      output = torch.nn.functional.softmax(output, dim=0)
    return output
 

SyntaxError: ignored

## Part 2 Attention Mechanism Extracting high level features

## Part 3 Sentence Generation

In [ ]:
import re
class AnnotationCleaner:
  def removePunctuation(self, annotation):
    # remove the punctuations
    return re.sub(r'[^\w\s]', '', test_str)

  def lower(self, annotation):
    # turn captions to lower
    return annotation.lower()
  
  def removeExtraSpace(self, annotation):
    # remove extra spaces
    new = re.sub(r" {2,}", " ", annotation)
    return new.strip()

  def __call__(self, annotation):
    output = self.removePunctuation(annotation)
    output = self.lower(annotation)
    output = self.removeExtraSpace(annotation)
    return output

In [ ]:
class Vocabularies:
  def __init__(self):
    self.word_to_idx = {}
    self.idx_to_word = {}
    self.len = 0

  def add_word(self, word):
    if word not in self.word_to_idx:
      self.word_to_idx[word] = self.len + 1
      self.len += 1
      self.idx_to_word[self.len] = word

We can use index to represent the words, but we can also use glove for the embedding

In [ ]:
# Build list of vocabs 
threshold = 4
def build_giant_list_of_vocabs(imageset):
  words = {}
  words_count = {}
  vocabs = Vocabularies()
  for img_id, image in imageset.images.items():
    for caption in image.annotations:
      for word in caption.split():
        if word not in words_count:
          words_count[word] = 1
        else:
          words_count[word] += 1
        if words_count[word] > threshold:
          words.add(word)
          vocab.add_word(word)
  # return a set of words, so that there would not be duplicates
  return words, words_count

In [ ]:
# download glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
!git clone https://github.com/stanfordnlp/glove
!cd glove && make

!pip install glove_python

In [ ]:
#Generate N-Grams
def n_grams(seq:Sequence[str], n:int) -> List:
    '''Extract all n-grams from a sequence

    An n-gram is a contiguous sub-sequence within `seq` of length `n`. This
    function extracts them (in order) from `seq`.

    Parameters
    ----------
    seq : sequence
        A sequence of words or token ids representing a transcription.
    n : int
        The size of sub-sequence to extract.

    Returns
    -------
    ngrams : list
    '''

    #Gets all n-grams in refernece input IN ORDER
    #Ex ref=friendship is magic
    #This returns [[friendship, is][is, magic]]

    total = []
    for sentence in seq: 
      i = 0
      ngrams = []
      while i+n <= len(sentence):
          ngrams.append(sentence[i:i+n])
          i+=1
      total.append(ngrams)
    return total


In [ ]:
#Bert Tokenizer
! pip install transformers
from transformers import BertTokenizer


def tokenize(data): 
  tokenized_inputs = bert_tokenizer(
      data,          # Input text
      add_special_tokens=True,  # add '[CLS]' and '[SEP]'
      padding='max_length',     # pad to a length specified by the max_length
      max_length=280,       # truncate all sentences longer than max_length -- max # of characters in Twitter
      return_tensors='pt',      # return everything we need as PyTorch tensors
  )

  input_ids = tokenized_inputs['input_ids']
  attention_masks = tokenized_inputs['attention_mask']
  return input_ids, attention_masks

In [ ]:
vocab_size = len(data['vocab']) # Number of vocabs
#from 413 PA1
def calculate_log_co_occurence(word_data, symmetric=False):
  "Compute the log-co-occurence matrix for our data."
  log_co_occurence = np.zeros((vocab_size, vocab_size))
  for input in word_data:
    # Note: the co-occurence matrix may not be symmetric
    log_co_occurence[input[0], input[1]] += 1
    log_co_occurence[input[1], input[2]] += 1
    log_co_occurence[input[2], input[3]] += 1
    # Diagonal entries are just the frequency of the word
    log_co_occurence[input[0], input[0]] += 1
    log_co_occurence[input[1], input[1]] += 1
    log_co_occurence[input[2], input[2]] += 1
    log_co_occurence[input[3], input[3]] += 1
    # If we want symmetric co-occurence can also increment for these.
    if symmetric:
      log_co_occurence[input[1], input[0]] += 1
      log_co_occurence[input[2], input[1]] += 1
      log_co_occurence[input[3], input[2]] += 1
  delta_smoothing = 0.5  # A hyperparameter.  You can play with this if you want.
  log_co_occurence += delta_smoothing  # Add delta so log doesn't break on 0's.
  log_co_occurence = np.log(log_co_occurence)
  return log_co_occurence

def loss_GloVe(W, W_tilde, b, b_tilde, log_co_occurence):
  """ Compute the GloVe loss given the parameters of the model. When W_tilde 
  and b_tilde are not given, then the model is symmetric (i.e. W_tilde = W,
  b_tilde = b). 

  Args:
    W: word embedding matrix, dimension V x d where V is vocab size and d
      is the embedding dimension
    W_tilde: for asymmetric GloVe model, a second word embedding matrix, with
      dimensions V x d
    b: bias vector, dimension V.
    b_tilde: for asymmetric GloVe model, a second bias vector, dimension V
    log_co_occurence: V x V log co-occurrence matrix (log X)
  
  Returns:
    loss: a scalar (float) for GloVe loss
  """
  n,_ = log_co_occurence.shape
  # Symmetric Case, no W_tilde and b_tilde
  if W_tilde is None and b_tilde is None:
    # Symmetric model
    ###########################   YOUR CODE HERE  ##############################
    A = np.matmul(W, W.T) + b*1 + b*1 - log_co_occurence
    loss = np.trace(np.matmul(A.T, A))
    ############################################################################
  else: 
    # Asymmetric model
    ###########################   YOUR CODE HERE  ##############################
    A = np.matmul(W, W_tilde.T) + b*1 + b_tilde*1 - log_co_occurence
    loss = np.trace(np.matmul(A.T, A))
    ############################################################################
  return loss

def grad_GloVe(W, W_tilde, b, b_tilde, log_co_occurence):
  """Return the gradient of GloVe objective w.r.t its parameters
  Args:
    W: word embedding matrix, dimension V x d where V is vocab size and d
      is the embedding dimension
    W_tilde: for asymmetric GloVe model, a second word embedding matrix, with
      dimensions V x d
    b: bias vector, dimension V.
    b_tilde: for asymmetric GloVe model, a second bias vector, dimension V
    log_co_occurence: V x V log co-occurrence matrix (log X)
  
  Returns:
    grad_W: gradient of the loss wrt W, dimension V x d
    grad_W_tilde: gradient of the loss wrt W_tilde, dimension V x d. Return 
      None if W_tilde is None.
    grad_b: gradient of the loss wrt b, dimension V x 1
    grad_b_tilde: gradient of the loss wrt b, dimension V x 1. Return 
      None if b_tilde is None.
  """
  n,_ = log_co_occurence.shape
  
  if W_tilde is None and b_tilde is None:
    # Symmmetric case
    ###########################   YOUR CODE HERE  ##############################
    A = np.matmul(W, W.T) + b*1 + b*1 - log_co_occurence
    grad_W =  2*np.matmul(A, W) + 2*np.matmul(A.T, W) 
    grad_b = 4*(np.matmul(A, np.ones(b.shape)))
    grad_W_tilde = None
    grad_b_tilde = None
    ############################################################################
  else:
    # Asymmetric case
    ###########################   YOUR CODE HERE  ##############################
    A = np.matmul(W, W_tilde.T) + b*1 + b_tilde*1 - log_co_occurence
    grad_W = 2*np.matmul(A, W_tilde)
    grad_W_tilde = 2*np.matmul(A.T, W)
    grad_b = 2*(np.matmul(A, np.ones(b.shape)))
    grad_b_tilde = 2*(np.matmul(A, np.ones(b_tilde.shape)))
    ############################################################################
    
  return grad_W, grad_W_tilde, grad_b, grad_b_tilde

def train_GloVe(W, W_tilde, b, b_tilde, log_co_occurence_train, log_co_occurence_valid, n_epochs, do_print=False):
  "Traing W and b according to GloVe objective."
  n,_ = log_co_occurence_train.shape
  learning_rate = 0.05 / n  # A hyperparameter.  You can play with this if you want.
  train_loss_list = np.zeros(n_epochs)
  valid_loss_list = np.zeros(n_epochs)
  vocab_size = log_co_occurence_train.shape[0]

  for epoch in range(n_epochs):
    grad_W, grad_W_tilde, grad_b, grad_b_tilde = grad_GloVe(W, W_tilde, b, b_tilde, log_co_occurence_train)
    W = W - learning_rate * grad_W
    b = b - learning_rate * grad_b
    if not grad_W_tilde is None and not grad_b_tilde is None:
      W_tilde = W_tilde - learning_rate * grad_W_tilde
      b_tilde = b_tilde - learning_rate * grad_b_tilde
    train_loss, valid_loss = loss_GloVe(W, W_tilde, b, b_tilde, log_co_occurence_train), loss_GloVe(W, W_tilde, b, b_tilde, log_co_occurence_valid)
    if do_print:
      print(f"Average Train Loss: {train_loss / vocab_size}, Average valid loss: {valid_loss / vocab_size}, grad_norm: {np.sum(grad_W**2)}")
    train_loss_list[epoch] = train_loss / vocab_size
    valid_loss_list[epoch] = valid_loss / vocab_size

  return W, W_tilde, b, b_tilde, train_loss_list, valid_loss_list

def generate_glove_embeddings(training, validation):
  """
  training: matrix of words
  validation: matrix of words
  """
  train_tokenize = tokenize(training)
  valid_tokenize = tokenize(validation)
  train4 = n_grams(train_tokenize, 4)
  valid4 = n_grams(valid_tokenize,4)
  train_co = log_co_occurence(train4)
  valid_co = log_co_occurence(valid4)

  init_variance = 0.05  # A hyperparameter.  You can play with this if you want.
  embedding_dim = 16
  W = init_variance * np.random.normal(size=(vocab_size, embedding_dim))
  W_tilde = init_variance * np.random.normal(size=(vocab_size, embedding_dim))
  b = init_variance * np.random.normal(size=(vocab_size, 1))
  b_tilde = init_variance * np.random.normal(size=(vocab_size, 1))

  return train_GloVe(W, W_tilde, b, b_tilde, train_co, valid_co, 
                     n_epochs=3, do_print=True)


In [ ]:
#Training + Helper functions for NMT Training

def string_to_index_list(s, char_to_index, end_token):
    """Converts a sentence into a list of indexes (for each character)."""
    return [char_to_index[char] for char in s] + [
        end_token
    ]  # Adds the end token to each index list

def visualize_attention(input_string, encoder, decoder, idx_dict, opts):
    """Generates a heatmap to show where attention is focused in each decoder step."""
    if idx_dict is None:
        line_pairs, vocab_size, idx_dict = load_data(opts["data_file_name"])
    char_to_index = idx_dict["char_to_index"]
    index_to_char = idx_dict["index_to_char"]
    start_token = idx_dict["start_token"]
    end_token = idx_dict["end_token"]

    max_generated_chars = 20
    gen_string = ""

    indexes = string_to_index_list(input_string, char_to_index, end_token)
    indexes = to_var(
        torch.LongTensor(indexes).unsqueeze(0), opts.cuda
    )  # Unsqueeze to make it like BS = 1

    encoder_annotations, encoder_hidden = encoder(indexes)

    decoder_hidden = encoder_hidden
    decoder_input = to_var(torch.LongTensor([[start_token]]), opts.cuda)  # For BS = 1
    decoder_inputs = decoder_input

    produced_end_token = False

    for i in range(max_generated_chars):
        ## slow decoding, recompute everything at each time
        decoder_outputs, attention_weights = decoder(
            decoder_inputs, encoder_annotations, decoder_hidden
        )
        generated_words = F.softmax(decoder_outputs, dim=2).max(2)[1]
        ni = generated_words.cpu().numpy().reshape(-1)  # LongTensor of size 1
        ni = ni[-1]  # latest output token

        decoder_inputs = torch.cat([decoder_input, generated_words], dim=1)

        if ni == end_token:
            break
        else:
            gen_string = "".join(
                [
                    index_to_char[int(item)]
                    for item in generated_words.cpu().numpy().reshape(-1)
                ]
            )

    if isinstance(attention_weights, tuple):
        ## transformer's attention mweights
        attention_weights, self_attention_weights = attention_weights

    all_attention_weights = attention_weights.data.cpu().numpy()

    for i in range(len(all_attention_weights)):
        attention_weights_matrix = all_attention_weights[i].squeeze()
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(attention_weights_matrix, cmap="bone")
        fig.colorbar(cax)

        # Set up axes
        ax.set_yticklabels([""] + list(input_string) + ["EOS"], rotation=90)
        ax.set_xticklabels(
            [""] + list(gen_string) + (["EOS"] if produced_end_token else [])
        )

        # Show label at every tick
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        # Add title
        plt.xlabel("Attention weights to the source sentence in layer {}".format(i + 1))
        plt.tight_layout()
        plt.grid("off")
        plt.show()

    return gen_string


def compute_loss(data_dict, encoder, decoder, idx_dict, criterion, optimizer, opts):
    """Train/Evaluate the model on a dataset.

    Arguments:
        data_dict: The validation/test word pairs, organized by source and target lengths.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model (with or without attention) to generate output tokens.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        criterion: Used to compute the CrossEntropyLoss for each decoder output.
        optimizer: Train the weights if an optimizer is given. None if only evaluate the model.
        opts: The command-line arguments.

    Returns:
        mean_loss: The average loss over all batches from data_dict.
    """
    start_token = idx_dict["start_token"]
    end_token = idx_dict["end_token"]
    char_to_index = idx_dict["char_to_index"]

    losses = []
    for key in data_dict:
        input_strings, target_strings = zip(*data_dict[key])
        input_tensors = [
            torch.LongTensor(string_to_index_list(s, char_to_index, end_token))
            for s in input_strings
        ]
        target_tensors = [
            torch.LongTensor(string_to_index_list(s, char_to_index, end_token))
            for s in target_strings
        ]

        num_tensors = len(input_tensors)
        num_batches = int(np.ceil(num_tensors / float(opts.batch_size)))

        for i in range(num_batches):

            start = i * opts.batch_size
            end = start + opts.batch_size

            inputs = to_var(torch.stack(input_tensors[start:end]), opts.cuda)
            targets = to_var(torch.stack(target_tensors[start:end]), opts.cuda)

            # The batch size may be different in each epoch
            BS = inputs.size(0)

            encoder_annotations, encoder_hidden = encoder(inputs)

            # The last hidden state of the encoder becomes the first hidden state of the decoder
            decoder_hidden = encoder_hidden

            start_vector = (
                torch.ones(BS).long().unsqueeze(1) * start_token
            )  # BS x 1 --> 16x1  CHECKED
            decoder_input = to_var(start_vector, opts.cuda)  # BS x 1 --> 16x1  CHECKED

            loss = 0.0

            seq_len = targets.size(1)  # Gets seq_len from BS x seq_len

            decoder_inputs = torch.cat(
                [decoder_input, targets[:, 0:-1]], dim=1
            )  # Gets decoder inputs by shifting the targets to the right

            decoder_outputs, attention_weights = decoder(
                decoder_inputs, encoder_annotations, decoder_hidden
            )
            decoder_outputs_flatten = decoder_outputs.view(-1, decoder_outputs.size(2))
            targets_flatten = targets.view(-1)

            loss = criterion(decoder_outputs_flatten, targets_flatten)

            losses.append(loss.item())

            ## training if an optimizer is provided
            if optimizer:
                # Zero gradients
                optimizer.zero_grad()
                # Compute gradients
                loss.backward()
                # Update the parameters of the encoder and decoder
                optimizer.step()

    return losses


def training_loop(
    train_dict, val_dict, idx_dict, encoder, decoder, criterion, optimizer, opts
):
    """Runs the main training loop; evaluates the model on the val set every epoch.
        * Prints training and val loss each epoch.
        * Prints qualitative translation results each epoch using TEST_SENTENCE
        * Saves an attention map for TEST_WORD_ATTN each epoch
        * Returns loss curves for comparison

    Arguments:
        train_dict: The training word pairs, organized by source and target lengths.
        val_dict: The validation word pairs, organized by source and target lengths.
        idx_dict: Contains char-to-index and index-to-char mappings, and start & end token indexes.
        encoder: An encoder model to produce annotations for each step of the input sequence.
        decoder: A decoder model (with or without attention) to generate output tokens.
        criterion: Used to compute the CrossEntropyLoss for each decoder output.
        optimizer: Implements a step rule to update the parameters of the encoder and decoder.
        opts: The command-line arguments.

    Returns:
        losses: Lists containing training and validation loss curves.
    """

    start_token = idx_dict["start_token"]
    end_token = idx_dict["end_token"]
    char_to_index = idx_dict["char_to_index"]

    loss_log = open(os.path.join(opts.checkpoint_path, "loss_log.txt"), "w")

    best_val_loss = 1e6
    train_losses = []
    val_losses = []

    mean_train_losses = []
    mean_val_losses = []

    early_stopping_counter = 0

    for epoch in range(opts.nepochs):

        optimizer.param_groups[0]["lr"] *= opts.lr_decay

        train_loss = compute_loss(
            train_dict, encoder, decoder, idx_dict, criterion, optimizer, opts
        )
        val_loss = compute_loss(
            val_dict, encoder, decoder, idx_dict, criterion, None, opts
        )

        mean_train_loss = np.mean(train_loss)
        mean_val_loss = np.mean(val_loss)

        if mean_val_loss < best_val_loss:
            checkpoint(encoder, decoder, idx_dict, opts)
            best_val_loss = mean_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter > opts.early_stopping_patience:
            print(
                "Validation loss has not improved in {} epochs, stopping early".format(
                    opts.early_stopping_patience
                )
            )
            print("Obtained lowest validation loss of: {}".format(best_val_loss))
            return (train_losses, mean_val_losses)

        gen_string = translate_sentence(TEST_SENTENCE, encoder, decoder, idx_dict, opts)
        print(
            "Epoch: {:3d} | Train loss: {:.3f} | Val loss: {:.3f} | Gen: {:20s}".format(
                epoch, mean_train_loss, mean_val_loss, gen_string
            )
        )

        loss_log.write("{} {} {}\n".format(epoch, train_loss, val_loss))
        loss_log.flush()

        train_losses += train_loss
        val_losses += val_loss

        mean_train_losses.append(mean_train_loss)
        mean_val_losses.append(mean_val_loss)

        save_loss_plot(mean_train_losses, mean_val_losses, opts)

    print("Obtained lowest validation loss of: {}".format(best_val_loss))
    return (train_losses, mean_val_losses)


def print_data_stats(line_pairs, vocab_size, idx_dict):
    """Prints example word pairs, the number of data points, and the vocabulary."""
    print("=" * 80)
    print("Data Stats".center(80))
    print("-" * 80)
    for pair in line_pairs[:5]:
        print(pair)
    print("Num unique word pairs: {}".format(len(line_pairs)))
    print("Vocabulary: {}".format(idx_dict["char_to_index"].keys()))
    print("Vocab size: {}".format(vocab_size))
    print("=" * 80)


def train(opts):
    line_pairs, vocab_size, idx_dict = load_data(opts["data_file_name"])
    print_data_stats(line_pairs, vocab_size, idx_dict)

    # Split the line pairs into an 80% train and 20% val split
    num_lines = len(line_pairs)
    num_train = int(0.8 * num_lines)
    train_pairs, val_pairs = line_pairs[:num_train], line_pairs[num_train:]

    # Group the data by the lengths of the source and target words, to form batches
    train_dict = create_dict(train_pairs)
    val_dict = create_dict(val_pairs)

    ##########################################################################
    ### Setup: Create Encoder, Decoder, Learning Criterion, and Optimizers ###
    ##########################################################################
    if opts.encoder_type == "rnn":
        encoder = GRUEncoder(
            vocab_size=vocab_size, hidden_size=opts.hidden_size, opts=opts
        )
    elif opts.encoder_type == "transformer":
        encoder = TransformerEncoder(
            vocab_size=vocab_size,
            hidden_size=opts.hidden_size,
            num_layers=opts.num_transformer_layers,
            opts=opts,
        )
    elif opts.encoder_type == "attention":
      encoder = AttentionEncoder(
            vocab_size=vocab_size,
            hidden_size=opts.hidden_size,
            opts=opts,
        )
    else:
        raise NotImplementedError

    if opts.decoder_type == "rnn":
        decoder = RNNDecoder(vocab_size=vocab_size, hidden_size=opts.hidden_size)
    elif opts.decoder_type == "rnn_attention":
        decoder = RNNAttentionDecoder(
            vocab_size=vocab_size,
            hidden_size=opts.hidden_size,
            attention_type=opts.attention_type,
        )
    elif opts.decoder_type == "transformer":
        decoder = TransformerDecoder(
            vocab_size=vocab_size,
            hidden_size=opts.hidden_size,
            num_layers=opts.num_transformer_layers,
        )
    elif opts.encoder_type == "attention":
      decoder = AttentionDecoder(
            vocab_size=vocab_size,
            hidden_size=opts.hidden_size,
        )
    else:
        raise NotImplementedError

    #### setup checkpoint path
    model_name = "h{}-bs{}-{}-{}".format(
        opts.hidden_size, opts.batch_size, opts.decoder_type, opts.data_file_name
    )
    opts.checkpoint_path = model_name
    create_dir_if_not_exists(opts.checkpoint_path)
    ####

    if opts.cuda:
        encoder.cuda()
        decoder.cuda()
        print("Moved models to GPU!")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        list(encoder.parameters()) + list(decoder.parameters()), lr=opts.learning_rate
    )

    try:
        losses = training_loop(
            train_dict, val_dict, idx_dict, encoder, decoder, criterion, optimizer, opts
        )
    except KeyboardInterrupt:
        print("Exiting early from training.")
        return encoder, decoder, losses

    return encoder, decoder, losses


def print_opts(opts):
    """Prints the values of all command-line arguments."""
    print("=" * 80)
    print("Opts".center(80))
    print("-" * 80)
    for key in opts.__dict__:
        print("{:>30}: {:<30}".format(key, opts.__dict__[key]).center(80))
    print("=" * 80)

In [ ]:
#Run Training 

TEST_SENTENCE = "the air conditioning is working"

rnn_args_s = AttrDict()
args_dict = {
    "data_file_name": "pig_latin_small",
    "cuda": True,
    "nepochs": 50,
    "checkpoint_dir": "checkpoints",
    "learning_rate": 0.005,
    "lr_decay": 0.99,
    "early_stopping_patience": 20,
    "batch_size": 64,
    "hidden_size": 32,
    "encoder_type": "rnn",  # options: rnn / transformer
    "decoder_type": "rnn",  # options: rnn / rnn_attention / transformer
    "attention_type": "",   # options: additive / scaled_dot
}
rnn_args_s.update(args_dict)

print_opts(rnn_args_s)
rnn_encode_s, rnn_decoder_s, rnn_losses_s = train(rnn_args_s)

translated = translate_sentence(
    TEST_SENTENCE, rnn_encode_s, rnn_decoder_s, None, rnn_args_s
)
print("source:\t\t{} \ntranslated:\t{}".format(TEST_SENTENCE, translated))

In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.Wix = nn.Linear(input_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)

        self.Wfx = nn.Linear(input_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)

        self.Wox = nn.Linear(input_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)

        self.Wcx = nn.Linear(input_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, m_prev, c_prev):
        """Forward pass of the LSTM computation for one time step.

        Arguments
            x: batch_size x input_size
            h_prev: batch_size x hidden_size
            c_prev: batch_size x hidden_size

        Returns:
            h_new: batch_size x hidden_size
            c_new: batch_size x hidden_size
        """
        i_t = torch.sigmoid(self.Wix(x) + self.Wim(m_prev))
        f_t = torch.sigmoid(self.Wfx(x) + self.Wfm(m_prev))
        o_t = torch.sigmoid(self.Wox(x) + self.Wom(m_prev))
        c_new = f_t * c_prev + i_t * torch.tanh(self.Wcx(x) + self.Wcm(m_prev))
        m_new = o_t * c_new
        p_new = torch.softmax(m_new)
        return m_new, c_new, p_new

In [ ]:
class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(LSTMDecoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = LSTMCell(input_size=hidden_size, hidden_size=hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs, annotations, hidden_init):
        """Forward pass of the non-attentional decoder RNN.

        Arguments:
            inputs: Input token indexes across a batch. (batch_size x seq_len)
            annotations: This is not used here. It just maintains consistency with the
                    interface used by the AttentionDecoder class.
            hidden_init: The hidden states from the last step of encoder, across a batch. (batch_size x hidden_size)

        Returns:
            output: Un-normalized scores for each token in the vocabulary, across a batch for all the decoding time steps. (batch_size x decoder_seq_len x vocab_size)
            None
        """
        batch_size, seq_len = inputs.size()
        embed = self.embedding(inputs)  # batch_size x seq_len x hidden_size

        hiddens = []
        h_prev = hidden_init

        for i in range(seq_len):
            x = embed[
                :, i, :
            ]  # Get the current time step input tokens, across the whole batch
            h_prev = self.rnn(x, h_prev)  # batch_size x hidden_size
            hiddens.append(h_prev)

        hiddens = torch.stack(hiddens, dim=1)  # batch_size x seq_len x hidden_size

        output = self.out(hiddens)  # batch_size x seq_len x vocab_size
        return output, None

In [ ]:
class MyRNNCell(nn.Module):
    def __init__(self, obs_dim, hidden_size):
        """Initialize RNN Cell."""
        super().__init__()

        self.obs_dim = obs_dim
        self.hidden_size = hidden_size
        

        # self.relu = nn.ReLU()
        # self.Ww = nn.Linear(obs_dim, obs_dim) # TODO: Check this

        # self.We = nn.Linear(obs_dim, hidden_size)

        # self.Wf = nn.Linear(hidden_size, hidden_size)

        # self.Wb = nn.Linear(hidden_size, hidden_size)

        # self.Wd = nn.Linear(hidden_size, hidden_size)

        self.Whi = 1 # Last layer of CNN
        self.Whx = nn.Linear(obs_dim, hidden_size)
        self.Whh = nn.Linear(hidden_size, hidden_size)
        self.Woh = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
    
    def forward(self, x, h_f, h_b, CNN_last_layer):
        """Compute forward pass for this RNN cell."""
        # x_t = self.Ww() # Some identity?

        # e_t = self.relu(self.We(x_t))

        # h_ft = self.relu(e_t + self.Wf(h_f))

        # h_bt = self.relu(e_t + self.Wb(h_b))

        # s_t = self.relu(self.Wd(h_ft + h_bt))

        b_v = self.Whi(CNN_last_layer)
        h_t = self.relu(self.Whx(x) + self.Whh(h_t_prev) + ) # TODO
        y_t = self.softmax(self.Woh(h_t) + )

        return s_t
    
class MyRNN(nn.Module):
    def __init__(self, obs_dim, hidden_size, output_dim):
        """Initialize RNN."""
        super().__init__()
        self.obs_dim = obs_dim
        self.hidden_size = hidden_size

        self.rnn_cell = MyRNNCell(obs_dim, hidden_size)

    def forward(self, x):
        """Compute forward pass on sequence x.
        
        Input sequence x has shape (B x L x D), where:
        B is batch size, L is sequence length, and D is the number of features.
        """
        batch_size, seq_len, n_feat = x.size()
        
        # Stores outputs of RNN cell
        output_arr = torch.zeros((batch_size, seq_len, self.output_dim))
        hidden_arr = torch.zeros((batch_size, seq_len, self.hidden_size))
        
        # Send to GPU. This is a gotcha, make sure to send Tensors created
        # in a model to the same device as input Tensors.
        output_arr = output_arr.float().to(x.device)
        hidden_arr = hidden_arr.float().to(x.device)

        hidden = self.init_hidden(batch_size, x.device)

        for i in range(seq_len):
            # For each iteration, compute RNN on input for current position
            output, hidden = self.rnn_cell(x[:, i, :], hidden)

            output_arr[:, i, :] = output
            hidden_arr[:, i, :] = hidden

        return output_arr, hidden_arr

    def init_hidden(self, batch_size, device):
        """Initialize RNN hidden state.
        
        Some people advocate for using random noise instead of zeros, or 
        training for the initial state. Personally, I don't know if it matters!
        """
        return torch.zeros(batch_size, self.hidden_size, device=device)

# Evaluation

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
# Example Usage of sentence_bleu
ref = [
    'this is moonlight'.split(),
    'Look, this is moonlight'.split(),
    'moonlight it is'.split()
]
test = 'it is moonlight'.split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))
 
test01 = 'it is cat and moonlight'.split()
print('BLEU score for test01-> {}'.format(sentence_bleu(ref, test01)))